In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from notebooks.ford.vins import load_vins

vins = load_vins()

Loaded 3060 VINs (filter_comment=None, include_openpilot=False, skipped=97)


Loaded AsBuilt data for 3060 VINs


In [5]:
import random

import pandas as pd

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.settings import VEHICLE_SETTINGS

pd.set_option('display.max_columns', None)

ecus = set(s.ecu[0] if type(s.ecu) == tuple else s.ecu for s in VEHICLE_SETTINGS)

rows = []
for vin in ('WF0NXXGCHNJR69712', *random.sample(vins, 9)):
  abd = AsBuiltData.from_vin(vin)
  rows.append([vin, *map(lambda s: abd.get_setting_value(s), VEHICLE_SETTINGS), *map(lambda ecu: abd.get_identifier(ecu, 0xF111), ecus)])

columns = [
  ('', 'VIN'),
  *map(lambda s: (f'{s.ecu[0].name} ({s.ecu[1]})' if type(s.ecu) == tuple else s.ecu.name, s.comment), VEHICLE_SETTINGS),
  *map(lambda ecu: (ecu.name, 'part'), ecus),
]

df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(columns))
df

ImageProcessingModuleA (14F403)                 \
                 VIN                         Vehicle Distance Alert   
0  WF0NXXGCHNJR69712                    C519 (Focus)            Off   
1  2FMPK4J94NBA89201                    CD539 (Edge)            Off   
2  1FTER4FH2LLA21452             P375 (Ranger Wave2)            Off   
3  3FTTW8E95NRA41751                 P758 (Maverick)            Off   
4  2FMPK4J95LBA34897                    CD539 (Edge)            Off   
5  1FTFW1E59NFC19594                         Missing        Missing   
6  1FMCU0BZ9MUA39421             CX482 (Escape/Kuga)            Off   
7  1FTFW1E84MKD96733                         Missing        Missing   
8  5LMCJ1D96KUL08917                         Missing        Missing   
9  3FA6P0SU8KR210988                         Missing        Missing   

                                                                  \
  Forward Collision Warning HUD High Beam Lane Departure Warning   
0                           Off       Off                     On   
1                           Off      AHBC                     On   
2                            On       Off                    Off   
3                           Off      AHBC                     On   
4                           Off      AHBC                     On   
5                       Missing   Missing                Missing   
6                           Off      AHBC                     On   
7                       Missing   Missing                Missing   
8                       Missing   Missing                Missing   
9                       Missing   Missing                Missing   

                                                                 \
  Lane Keeping Aid Driver Alert System Traffic Sign Recognition   
0               On                 Off                       On   
1               On                  On                      Off   
2               On                 Off                      Off   
3               On                  On                      Off   
4               On                  On                      Off   
5          Missing             Missing                  Missing   
6               On                  On                      Off   
7          Missing             Missing                  Missing   
8          Missing             Missing                  Missing   
9          Missing             Missing                  Missing   

                                                                             \
  Traffic Sign Recognition Mode Traffic Sign Recognition Feature LKS Switch   
0                   Camera Only                             SLIF    Network   
1                        Fusion                              Off    Network   
2                        Fusion                              Off    Network   
3                        Fusion                              Off    Network   
4                        Fusion                              Off    Network   
5                       Missing                          Missing    Missing   
6                        Fusion                              Off    Network   
7                       Missing                          Missing    Missing   
8                       Missing                          Missing    Missing   
9                       Missing                          Missing    Missing   

                                                                        \
  Pre-Collision Assist Type Enable ACC Driving Side Region and Country   
0              Radar Fusion        Off         Left               0x25   
1               Camera Only        Off        Right               0x75   
2               Camera Only        Off        Right               0x75   
3               Camera Only        Off        Right               0x75   
4               Camera Only        Off        Right               0x75   
5                   Missing    Missing      Missing            Missing   
6    

In [4]:
from notebooks.ford.settings import VehicleSettings

important_settings = [
  # Q3
  VehicleSettings.ipma_vehicle,
  # VehicleSettings.ipma_driving_side,
  VehicleSettings.ipma_enable_adaptive_cruise,
  VehicleSettings.ipma_enable_traffic_jam_assist,
  VehicleSettings.ipma_shift_by_wire,
  VehicleSettings.ipma_speed_limit_sign_unit,

  # Q4
  VehicleSettings.ipma_vehicle_cfg_vehicle_type,
  VehicleSettings.ipma_vehicle_cfg_acc_type,
  VehicleSettings.ipma_module_feature_cfg_tja,
  VehicleSettings.ipma_vehicle_cfg_gear_shift_by_wire,
  VehicleSettings.ipma_market_cfg_speed_limit_sign_unit,
  # VehicleSettings.ipma_market_cfg_region,
  # VehicleSettings.ipma_market_cfg_country,
  # VehicleSettings.ipma_market_cfg_driving_side,

  VehicleSettings.ipc_display_units,

  VehicleSettings.pscm_traffic_jam_assist,
  VehicleSettings.pscm_lane_centering_assist,

  VehicleSettings.abs_cruise_control_mode,
  VehicleSettings.abs_steering_gear,

  VehicleSettings.apim_country_code_first_letter,
  VehicleSettings.apim_country_code_second_letter,
  VehicleSettings.apim_lane_change_assist,
  VehicleSettings.apim_adaptive_cruise_mode,
  VehicleSettings.apim_transmission_type,
  VehicleSettings.apim_fuel_type,
  VehicleSettings.apim_wheel_base,
  VehicleSettings.apim_wheel_base_cont,
  VehicleSettings.apim_vehicle_weight,

  VehicleSettings.apim_sync4_country_code_first_letter,
  VehicleSettings.apim_sync4_country_code_second_letter,
  VehicleSettings.apim_sync4_lane_change_assist,
  VehicleSettings.apim_sync4_adaptive_cruise,
  VehicleSettings.apim_sync4_transmission_type,
  VehicleSettings.apim_sync4_fuel_type,
  VehicleSettings.apim_sync4_wheel_base,
  VehicleSettings.apim_sync4_wheel_base_cont,
  VehicleSettings.apim_sync4_vehicle_weight,
]

rows = []
for vin in ('WF0NXXGCHNJR69712', 'WF0NXXGCHNKD75963', 'RHAETAGEXN2000154', *random.sample(vins, 19)):
  abd = AsBuiltData.from_vin(vin)
  rows.append([vin, *map(lambda s: abd.get_setting_value(s), important_settings)])

columns = [('', 'VIN'), *map(lambda s: (f'{s.ecu[0].name} ({s.ecu[1]})' if type(s.ecu) == tuple else s.ecu.name, s.comment), important_settings)]

df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(columns))
df

ImageProcessingModuleA (14F403)                          \
                  VIN                         Vehicle   Enable ACC Enable TJA   
0   WF0NXXGCHNJR69712                    C519 (Focus)          Off   Disabled   
1   WF0NXXGCHNKD75963                    C519 (Focus)          Off   Disabled   
2   RHAETAGEXN2000154             CX482 (Escape/Kuga)  RadarFusion    Enabled   
3   2FMPK4J93RBA06010                    CD539 (Edge)          Off   Disabled   
4   3FTTW8F98PRA63972                 P758 (Maverick)          Off   Disabled   
5   3FA6P0LU8KR136034                         Missing      Missing    Missing   
6   2FMPK4K93KBC59431                    CD539 (Edge)  RadarFusion    Enabled   
7   1FM5K8FW9MNA19595                 U625 (Explorer)  RadarFusion    Enabled   
8   1FM5K8FW2MNA03819                 U625 (Explorer)  RadarFusion    Enabled   
9   1FM5K8FW9MNA02151                 U625 (Explorer)  RadarFusion    Enabled   
10  MAJ6P1WL2JC248821                         Missing      Missing    Missing   
11  2FMPK4AP7LBB43828                    CD539 (Edge)  RadarFusion    Enabled   
12  3FMCR9G60PRD23746     CX430 (Bronco Sport OnRoad)          Off   Disabled   
13  1FMSK8FHXNGA83042                 U625 (Explorer)  RadarFusion    Enabled   
14  1FMSK7FH3NGA32458                 U625 (Explorer)  RadarFusion    Enabled   
15  MAJ3S2KE7LC345837                         Missing      Missing    Missing   
16  3FTTW8E3XNRA75034                 P758 (Maverick)          Off   Disabled   
17  3FTTW8H30RRA15636                 P758 (Maverick)          Off   Disabled   
18  1FMCU9HDXKUC15096                         Missing      Missing    Missing   
19  1FMCU9J96NUA78457             CX482 (Escape/Kuga)  RadarFusion    Enabled   
20  1FATP8UH8L5161359                         Missing      Missing    Missing   
21  5LM5J7WC5PGL01886          U611 (Lincoln Aviator)  RadarFusion    Enabled   

                                       ImageProcessingModuleA (14H107)  \
   Shift by Wire Speed Limit Sign Unit          VehicleCfg_VehicleType   
0            Off                   MPH                         Missing   
1            Off                   KPH                         Missing   
2             On                   KPH                         Missing   
3             On                   MPH                         Missing   
4             On                   MPH                         Missing   
5        Missing               Missing                         Missing   
6             On                   MPH                         Missing   
7             On                   MPH                         Missing   
8             On                   KPH                         Missing   
9             On                   MPH                         Missing   
10       Missing               Missing                         Missing   
11            On                   MPH                         Missing   
12            On                   MPH                         Missing   
13            On                   MPH                         Missing   
14            On                   MPH                         Missing   
15       Missing               Missing                         Missing   
16            On                   MPH                         Missing   
17            On                   MPH                         Missing   
18       Missing               Missing                         Missing   
19            On                   MPH                         Missing   
20       Missing               Missing                         Missing   
21            On                   MPH                         Missing   

                                                                       \
   VehicleCfg_ACCType ModuleFeatureCfg_TJA VehicleCfg_GearShiftByWire   
0             Missing              Missing                    Missing   
1             Missing              Missing                    Missing   
2       